# Практическая работа №2

## Импорт необходимых библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from csv import DictWriter

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_classif, RFE, SelectKBest, chi2, SelectFromModel
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.ticker import MultipleLocator
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from functions import plot_roc_curves, plot_bar_graph
from typing import Dict

In [2]:
dataframe_train = pd.read_csv("../data/train.csv")
dataframe_test = pd.read_csv("../data/test.csv")
dataframe_train.head(10)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,213892,Male,48,1,35.0,1,1-2 Year,No,24922.0,26.0,86,0
1,269011,Male,58,1,28.0,0,1-2 Year,Yes,63541.0,26.0,57,0
2,31464,Male,44,1,28.0,1,1-2 Year,No,30027.0,26.0,286,0
3,86379,Female,22,1,8.0,0,< 1 Year,Yes,35039.0,152.0,180,0
4,456843,Male,24,1,6.0,1,< 1 Year,No,24149.0,152.0,17,0
5,195890,Female,49,1,28.0,0,1-2 Year,Yes,2630.0,157.0,156,0
6,111681,Male,28,1,6.0,1,< 1 Year,No,31154.0,152.0,150,0
7,427068,Male,22,1,12.0,1,< 1 Year,No,27956.0,152.0,174,0
8,337519,Female,25,1,14.0,1,< 1 Year,No,32658.0,152.0,32,0
9,12666,Male,47,1,0.0,0,1-2 Year,Yes,2630.0,26.0,79,0


Выведем информацию о таблицах.

In [3]:
dataframe_train.info()
dataframe_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229292 entries, 0 to 229291
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    229292 non-null  int64  
 1   Gender                229292 non-null  object 
 2   Age                   229292 non-null  int64  
 3   Driving_License       229292 non-null  int64  
 4   Region_Code           229292 non-null  float64
 5   Previously_Insured    229292 non-null  int64  
 6   Vehicle_Age           229292 non-null  object 
 7   Vehicle_Damage        229292 non-null  object 
 8   Annual_Premium        229292 non-null  float64
 9   Policy_Sales_Channel  229292 non-null  float64
 10  Vintage               229292 non-null  int64  
 11  Response              229292 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 21.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152862 entries, 0 to 152861
Data columns (total 

In [4]:
dataframe_train.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

Из вывода видно, что в столбцах нет пустых значений. Разделим наши признаки на категориальные и числовые.

In [5]:
dataframe_categorical = ["Gender", "Vehicle_Age", "Vehicle_Damage"]
dataframe_numerical = ["Age", "Driving_License", "Previously_Insured", "Region_Code", "Annual_Premium", "Policy_Sales_Channel", "Vintage"]
dataframe_all = dataframe_numerical + dataframe_categorical
result_column = "Response"

Для всей работы выберем 20000 элементов из датафрейма, используя метод сэмплирования из библиотеки Pandas.

In [6]:
number_of_elements = 20000  # Количество элементов
dataframe_train = dataframe_train.sample(n=number_of_elements, replace = False)  # replace: флаг, указывающий, следует ли использовать повторения при выборке.
dataframe_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 185354 to 127126
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    20000 non-null  int64  
 1   Gender                20000 non-null  object 
 2   Age                   20000 non-null  int64  
 3   Driving_License       20000 non-null  int64  
 4   Region_Code           20000 non-null  float64
 5   Previously_Insured    20000 non-null  int64  
 6   Vehicle_Age           20000 non-null  object 
 7   Vehicle_Damage        20000 non-null  object 
 8   Annual_Premium        20000 non-null  float64
 9   Policy_Sales_Channel  20000 non-null  float64
 10  Vintage               20000 non-null  int64  
 11  Response              20000 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 2.0+ MB


In [7]:
number_of_elements = 2000
sns.pairplot(dataframe_train.sample(n=2000)[dataframe_all + [result_column]], hue=result_column, corner=True, markers=["o", "D"])

Так как признак id не играет важной роли в предсказании, то можем удалить его из датафрейма.

In [8]:
dataframe_train.drop("id", axis=1, inplace=True)

In [9]:
for column in dataframe_categorical:
    print(dataframe_train[column].unique())

['Female' 'Male']
['< 1 Year' '1-2 Year' '> 2 Years']
['No' 'Yes']


In [10]:
dataframe_train[dataframe_numerical].describe()

,Age,Driving_License,Previously_Insured,Region_Code,Annual_Premium,Policy_Sales_Channel,Vintage
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,38.367800,0.997850,0.496250,26.441650,30833.563600,112.483150,155.744400
std,15.174051,0.046319,0.499998,13.162255,18097.663459,54.214181,83.670237
min,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000
25%,25.000000,1.000000,0.000000,15.000000,24571.250000,29.000000,83.000000
50%,35.000000,1.000000,0.000000,28.000000,31762.000000,151.000000,156.000000
75%,48.000000,1.000000,1.000000,35.000000,39399.000000,152.000000,229.000000
max,85.000000,1.000000,1.000000,52.000000,540165.000000,163.000000,299.000000


Произведём кодирование категориальных признаков

In [11]:
label_encoder = LabelEncoder()
for column in dataframe_categorical:
    dataframe_train[column] = label_encoder.fit_transform(dataframe_train[column])
    dataframe_test[column] = label_encoder.fit_transform(dataframe_test[column])
    dataframe_train[column] = dataframe_train[column].astype("category")
    dataframe_test[column] = dataframe_test[column].astype("category")
    print(dataframe_train[column].unique())

[0, 1]
Categories (2, int64): [0, 1]
[1, 0, 2]
Categories (3, int64): [0, 1, 2]
[0, 1]
Categories (2, int64): [0, 1]


Произведём нормирование всех числовых признаков

In [12]:
min_max_scaler = MinMaxScaler()
dataframe_train[dataframe_numerical] = min_max_scaler.fit_transform(dataframe_train[dataframe_numerical])
dataframe_test[dataframe_numerical] = min_max_scaler.fit_transform(dataframe_test[dataframe_numerical])
dataframe_train.describe()

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,0.282582,0.997850,0.508493,0.496250,0.052468,0.688168,0.504306,0.159100
std,0.233447,0.046319,0.253120,0.499998,0.033668,0.334655,0.289516,0.365778
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.076923,1.000000,0.288462,0.000000,0.040818,0.172840,0.252595,0.000000
50%,0.230769,1.000000,0.538462,0.000000,0.054196,0.925926,0.505190,0.000000
75%,0.430769,1.000000,0.673077,1.000000,0.068403,0.932099,0.757785,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Отбор информативных признаков
Отбор информативных признаков - это процесс выбора наиболее значимых признаков из набора данных для использования в модели машинного обучения. Он помогает улучшить качество модели и сократить время её обучения. Если в наборе данных есть много признаков, которые не имеют большого влияния на результаты моделирования, то отбрасывание этих признаков улучшит качество модели. Существует несколько методов отбора информативных признаков, таких как обертывание, фильтрация и встроенные методы.

Этот код разделяет данные на обучающий и тестовый наборы с помощью функции train_test_split из библиотеки sklearn.model_selection. Функция train_test_split разделяет данные на обучающий и тестовый наборы в соотношении 70/30.

In [13]:
x = dataframe_train[dataframe_all]
y = dataframe_train[result_column]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=7)
number_best_features = 5

### Метод фильтрации
Метод фильтрации - это метод отбора признаков на основе их статистических свойств. Он основан на оценке значимости каждого признака по отдельности и выборе наиболее значимых признаков. Примерами таких методов можно назвать корреляционный анализ и анализ дисперсии.
В этом коде используется метод SelectKBest из библиотеки scikit-learn для выбора лучших признаков из набора данных. Переменная skb создает объект SelectKBest с параметром k=number_best_features, указывающий количество наилучших признаков, которые будут выбраны. Метод fit() применяется к skb и используется для обучения модели на данных x и y. Метод get_feature_names_out() возвращает список лучших признаков.

In [14]:
skb = SelectKBest(score_func=chi2, k=number_best_features)  # SelectKBest использует ANOVA F-значение для оценки важности каждого признака и выбирает лучшие признаки на основе этой оценки.
top_five = skb.fit(x, y)  # Используется для обучения модели на данных x и целевой переменной y.
list_of_best_features = top_five.get_feature_names_out()
print(list_of_best_features)

['Age' 'Previously_Insured' 'Policy_Sales_Channel' 'Vehicle_Age'
 'Vehicle_Damage']


### Метод обёртки
Метод обертывания - это метод отбора признаков на основе обучения модели машинного обучения с использованием различных наборов признаков. Он основан на оценке качества модели для каждого набора признаков и выборе наилучшего набора.
В коде используется метод обертывания Recursive Feature Elimination из библиотеки scikit-learn для выбора лучших признаков из набора данных. Переменная clf создает объект LinearRegression(), который используется в качестве модели машинного обучения.

Переменная rfe создает объект RFE с параметрами clf, n_features_to_select=n_features_select и step=1. Параметр number_best_features указывает количество лучших признаков, которые будут выбраны. Параметр step указывает шаг уменьшения количества признаков на каждой итерации. Метод fit() применяется к объекту rfe и используется для обучения модели на данных x и y. Метод get_feature_names_out() возвращает список лучших признаков.

In [15]:
lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=number_best_features)
top_five = rfe.fit(x, y)
selected_features_RFE = top_five.get_feature_names_out()
print(selected_features_RFE)

['Driving_License' 'Previously_Insured' 'Annual_Premium'
 'Policy_Sales_Channel' 'Vehicle_Damage']


### Встроенные методы
Встроенные методы - это методы отбора признаков, которые включены в процесс обучения модели машинного обучения. Они основаны на оценке значимости каждого признака в процессе обучения модели и выборе наиболее значимых признаков.

In [16]:
selector = SelectFromModel(estimator=lr, max_features=number_best_features)  # Мета-оценщик. Используется для извлечения лучших признаков из заданного набора данных в соответствии с важностью весов.
# Параметр estimator - оценщик, который используется для выбора признаков
top_five = selector.fit(x, y)
selected_features_built_in = top_five.get_feature_names_out()
print(selected_features_built_in)

['Age' 'Driving_License' 'Previously_Insured' 'Annual_Premium'
 'Vehicle_Damage']


### Сравнение полученных результатов
Используем метод Gaussian Naive Bayes из библиотеки Scikit-learn для обучения модели машинного обучения на данных x_train и y_train. Он реализует гауссовский наивный байесовский алгоритм для классификации. Переменная model создает объект GaussianNB() и применяет к нему метод fit() с параметрами x_train и y_train, это обучает модель на данных. Переменная masks хранит список масок, каждая из которых содержит True для каждого признака в x_train. Переменная scoring хранит список оценок ROC-AUC для каждой маски. Метод predict_proba() применяется к модели и используется для предсказания вероятности классов на данных x_test. Метод roc_auc_score() вычисляет оценку ROC-AUC для предсказанных вероятностей и фактических значений y_test.

Рассчитаем значение оценки для изначального фрейма данных.

In [17]:
masks = scoring = tuple()
model = GaussianNB().fit(x_train, y_train)
masks += ([True for _ in range(x_train.shape[1])],)
scoring += (roc_auc_score(y_test, model.predict_proba(x_test)[:, 1]),)
print("ROC-AUC: " + str(round(scoring[-1], 3)))

ROC-AUC: 0.867


Создаём словарь values_dict, который содержит три метода отбора признаков: chi2, rfe и built-in, проходим по каждому методу отбора признаков и выполняем подсчет оценки. Создаём объект GaussianNB() и применяем к нему метод fit() с параметрами selector.transform(x_train) и y_train. Это обучит модель на данных, отобранных методом selector.

In [18]:
values_dict = {
    "chi2": skb,
    "rfe": rfe,
    "built-in": selector,
}

for selector_name, selector in values_dict.items():
    model = GaussianNB().fit(selector.transform(x_train), y_train)
    masks += (selector.get_support(),)
    scoring = (roc_auc_score(y_test, model.predict_proba(selector.transform(x_test))[:, 1]),)
    print("ROC-AUC: " + str(round(scoring[-1], 3)))

ROC-AUC: 0.868
ROC-AUC: 0.85
ROC-AUC: 0.867


Исходя из результатов можно сделать вывод, что третий метод отбора признаков является наилучшим.

## Удаление незначимых признаков

In [19]:
dataframe_train = dataframe_train.loc[:, np.append(selected_features_built_in, result_column)]  # loc используется для выборки строк и столбцов
dataframe_test = dataframe_test.loc[:, selected_features_built_in]

dataframe_all = selected_features_built_in

x = dataframe_train[dataframe_all]
y = dataframe_train[result_column]

In [20]:
dataframe_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 185354 to 127126
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Age                 20000 non-null  float64 
 1   Driving_License     20000 non-null  float64 
 2   Previously_Insured  20000 non-null  float64 
 3   Annual_Premium      20000 non-null  float64 
 4   Vehicle_Damage      20000 non-null  category
 5   Response            20000 non-null  int64   
dtypes: category(1), float64(4), int64(1)
memory usage: 957.2 KB


## Сэмплирование данных
Семплирование данных - это метод корректировки обучающей выборки с целью балансировки распределения классов в исходном наборе данных.

In [21]:
# sampling = ((x, y), RandomOverSampler().fit_resample(x, y), RandomUnderSampler().fit_resample(x, y))

over_sampler = RandomOverSampler()
under_sampler = RandomUnderSampler()

x_over, y_over = over_sampler.fit_resample(x, y)
x_under, y_under = under_sampler.fit_resample(x, y)

scoring = tuple()

print("ROC-AUC:")

for sx, sy in [(x, y), (x_over, y_over), (x_under, y_under)]:
    scoring += (round(max(cross_val_score(GaussianNB(), sx, sy, scoring="roc_auc", n_jobs=-1)), 3), )
    print(scoring[-1])

ROC-AUC:
0.866
0.865
0.869


Самое лучшее значение получилось у набора, где был under-sampling.

In [22]:
x, y = x_under, y_under
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=7)  # random_state устанавливает начальное значение для генератора случайных чисел.

## Построение моделей классификаторов
Построение моделей классификаторов - это процесс создания алгоритма машинного обучения, который может классифицировать данные на основе определенных признаков. Есть множество методов и алгоритмов для построения моделей классификаторов в зависимости от типа данных и задачи классификации:
1. Логистическая регрессия
2. Метод ближайших соседей
3. Наивный байесовский классификатор
4. Дискриминантный анализ (линейный дискриминантный анализ,
квадратичный дискриминантный анализ)
5. Машина опорных векторов.

Напишем функцию для создания и обучения модели. algorythm - алгоритм, с помощью которого мы будем обучать нашу модель. param_grid - это словарь, который содержит набор параметров для модели машинного обучения. Каждый ключ в словаре соответствует имени параметра, а значение - списку возможных значений параметра. GridSearchCV будет перебирать все возможные комбинации значений параметров из словаря param_grid и выбирать ту комбинацию, которая дает наилучший результат на кросс-валидации. Обучаем модель на данных x_train, y_train и оцениваем на тестовых данных x_test и y_test с помощью метрики ROC-AUC.

In [23]:
def create_train_model(algorythm, param_grid=None) -> GridSearchCV:
    if param_grid is None:
        param_grid = {}
    print("Создание и обучение модели...")
    """Создать и обучить модель с подбором параметров."""
    new_model = GridSearchCV(algorythm, param_grid, n_jobs=-1, scoring="roc_auc").fit(x_train, y_train)
    if param_grid:
        print(new_model.best_estimator_)
    print("ROC-AUC: " + str(round(new_model.score(x_test, y_test), 6)))
    return new_model

### Логистическая регрессия
Логистическая регрессия - статистическая модель, используемая для прогнозирования вероятности возникновения некоторого события путём его сравнения с логистической кривой.

In [24]:
param_grid = {
    "penalty": ["l1", "l2"], # C добавить
    "solver": ["saga", "liblinear"],
    "max_iter": range(1, 10)
}

lg = create_train_model(LogisticRegression(), param_grid)

Создание и обучение модели...
LogisticRegression(max_iter=7, penalty='l1', solver='liblinear')
ROC-AUC: 0.855604


C:\Users\nikit\anaconda3\envs\2_lab_test\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Метод ближайших соседей

In [25]:
param_grid = {
    "n_neighbors": range(15, 20),  # от 10 до 30
    "weights": ["uniform", "distance"]
    # добавить раст.
}

knc = create_train_model(KNeighborsClassifier(), param_grid)

Создание и обучение модели...
KNeighborsClassifier(n_neighbors=19)
ROC-AUC: 0.884846


### Наивный байесовский классификатор
Наивный байесовский классификатор - это алгоритм классификации, основанный на теореме Байеса.

In [26]:
params =dict()
gnb = create_train_model(GaussianNB(), params)

Создание и обучение модели...
GaussianNB()
ROC-AUC: 0.862634


### Линейный дискриминантный анализ

In [27]:
params = {'solver': ['lsqr', 'eigen'], 'shrinkage': [None, 'auto', 0.5, 0.9]}
lda = create_train_model(LinearDiscriminantAnalysis(), params)

Создание и обучение модели...
LinearDiscriminantAnalysis(shrinkage='auto', solver='lsqr')
ROC-AUC: 0.852332


### Квадратичный дискриминантный анализ

In [28]:
params =  {'store_covariance': [True, False], 'reg_param': [0, 0.1, 0.25, 0.5, 0.7, 1]}
qda = create_train_model(QuadraticDiscriminantAnalysis(), params)

Создание и обучение модели...
QuadraticDiscriminantAnalysis(reg_param=0, store_covariance=True)
ROC-AUC: 0.870164


### Машина опорных векторов

In [29]:
params = {'max_iter': range(1, 10)}
# svc = create_train_model(SVC(probability=True))

## Построение ROC кривых и столбчатой диаграммы score

In [30]:
models = (lg, knc, gnb, lda, qda)
names = scoring = probs = tuple()
for model in models:
    names += (model.best_estimator_.__class__.__name__,)
    scoring += (round(model.score(x_test, y_test), 3),)
    probs += (model.predict_proba(x_test)[:, 1],)

(array([0.68022258, 0.00478796, 0.71957194, ..., 0.70391118, 0.82886406,
       0.73026736]), array([0.78947368, 0.        , 0.63157895, ..., 0.78947368, 0.89473684,
       0.78947368]), array([9.96096895e-001, 1.05789353e-116, 9.97027147e-001, ...,
       9.95196741e-001, 9.98073368e-001, 9.96783811e-001]), array([0.78836782, 0.04148822, 0.82216016, ..., 0.81600334, 0.90776116,
       0.83397341]), array([9.83823405e-001, 1.51759466e-106, 9.81875649e-001, ...,
       9.90347203e-001, 9.95291348e-001, 9.86221561e-001]))


In [31]:
fig, axes = plt.subplots(1, 2)
fig.set_figwidth(18)
plot_roc_curves(axes[0], probs, names, y_test, axes)
plot_bar_graph(axes[1], names, scoring)
plt.show()

In [32]:
best_model = models[scoring.index(max(scoring))]
print("В результате анализа была выбрана модель: " + str(best_model.best_estimator_))

В результате анализа была выбрана модель: KNeighborsClassifier(n_neighbors=19)


Записываем предсказания наилучшей модели в файл

In [37]:
with open("../data/submission.csv", "w", encoding="utf-8") as csv:
    writer = DictWriter(
        csv, delimiter=",", lineterminator="\r",
        fieldnames=["PassengerId", "Survived"]
    )
    writer.writeheader()

    dataframe_test = dataframe_test.loc[:, list_of_best_features]
    prediction = best_model.predict(dataframe_test)

    for i, predict in enumerate(prediction):
        writer.writerow({"PassengerId": test_ids[i], "Survived": predict})

NameError: name 'test_ids' is not defined